## First step to clean the dataset:

- Select only English movies
- Select onlu full-length feature (movies of more than one hour long)
- Remove movies for which we don't have the title

In [121]:
import pandas as pd
#import requests
import numpy as np
import string
import json

In [122]:
character_metadata = pd.read_csv('Data/character.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_release_date',
                                                                              'Character_name',
                                                                              'Actor_date_of_birth',
                                                                              'Actor_gender',
                                                                              'Actor_height',
                                                                              'Actor_ethnicity',
                                                                              'Actor_name',
                                                                              'Actor_age_at_movie_release',
                                                                              'Freebase_character/actor_map_ID',
                                                                              'Freebase_character_ID',
                                                                              'Freebase_actor_ID'])

movie_metadata = pd.read_csv('Data/movie.metadata.tsv', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'Freebase_movie_ID',
                                                                              'Movie_name',
                                                                              'Movie_release_date',
                                                                              'Movie_box_office_revenue',
                                                                              'Movie_runtime',
                                                                              'Movie_languages',
                                                                              'Movie_countries',
                                                                              'Movie_genres'])

plot_summaries = pd.read_csv('Data/plot_summaries.txt', sep='\t', names = ['Wikipedia_movie_ID',
                                                                              'summary'])

In [123]:
#Copy of movie_metadata dataset
movie_metadata_filtered = movie_metadata.copy(deep= True)

#Filtering of movie_metada, keeping only full-length features movies (more than 60 minutes long), in English and with that have a Title in the database.
movie_metadata_filtered = movie_metadata_filtered[movie_metadata_filtered.Movie_languages.str.contains("English")]
movie_metadata_filtered.dropna(subset=['Movie_name'],inplace=True)
movie_metadata_filtered = movie_metadata_filtered[movie_metadata_filtered.Movie_runtime >= 60]

print(movie_metadata_filtered)

       Wikipedia_movie_ID Freebase_movie_ID  \
0                  975900         /m/03vyhn   
1                 3196793         /m/08yl5d   
3                 9363483        /m/0285_cd   
6                18998739        /m/04jcqvw   
7                10408933        /m/02qc0j7   
...                   ...               ...   
81735            32468537        /m/0crwd9y   
81736            35228177        /m/0j7hxnt   
81737            34980460        /m/0g4pl34   
81738             9971909        /m/02pygw1   
81740            12476867        /m/02w7zz8   

                                              Movie_name Movie_release_date  \
0                                         Ghosts of Mars         2001-08-24   
1      Getting Away with Murder: The JonBenét Ramsey ...         2000-02-16   
3                                       White Of The Eye               1987   
6                              The Sorcerer's Apprentice               2002   
7                               Alexande

In [124]:
movie_metadata_filtered.Movie_name[movie_metadata_filtered.Movie_name.str.contains(" la ")].head()

#There are still some Title that are not in English...

21500                              Murder a la Mod
33420    La pelota vasca. La piel contra la piedra
39681                             Café de la plage
40550                         Les Damnés de la mer
45773               Lumumba, la mort d'un prophète
Name: Movie_name, dtype: object

In [125]:
movie_English_only = movie_metadata_filtered[movie_metadata_filtered.Movie_languages == '{"/m/02h40lc": "English Language"}']

In [126]:
movie_English_only.Movie_name[movie_English_only.Movie_name.str.contains(" la ")].head()

21500    Murder a la Mod
70558      Irma la Douce
Name: Movie_name, dtype: object

In [127]:
movie_English_only.to_csv(path_or_buf = "Data/movies_metadata_english_only.csv", sep=',')

In [128]:
# IMDB files 

ratings_imdb = pd.read_table('Data/ratings_imdb.tsv', sep="\t")
ratings_imdb.head()


movies_imdb= pd.read_table('Data/movies_imdb.tsv', sep="\t")



In [161]:
# Genre processing
# Objectif: donner des noms de genre communs pour tous les films
#(grosse) Inspiration du projet adamax 2022 (spliter + les genres ou en faire moins?)

### 1st step: create a new genre dictionnary /!\ they did it themselves, should we do it as well?
dict_genre = {
    "Porn": ["Pornographic movie", "Pornography", "Softcore Porn", "Adult", "Sexploitation", "Erotica", "homoeroticism"], 
    "Action": ["Action/adventure", "ninja movie", "epic", "auto racing", "martial arts film", "race movie", "samurai cinema"],
    "Adventure": ["Adventure", "travel"],
    "Animals": ["Animal Picture"],
    "Anime": [],
    "Alien Film": ["Alien invasion"],
    "Animation": ["Animated cartoon"],
    "Beach Film": ["Beach Party film"],
    "Biographical film": ["Biography", "Biopic [feature]"],
    "Buddy film": ["Buddy Picture", "Buddy cop"],
    "Coming of age": ["Coming-of-age film"],
    "Comedy": ["Humour", "Tragicomedy", "Ealing Comedies", "Comdedy", "Adventure Comedy", "Commedy of manners", "Workplace comedy", "comedy-drama"], 
    "Crime": ['crime', 'crime fiction', 'crime thriller', "master criminal films", "detective"],
    "Documentary": ["Political Documetary", "Docudrama", "Historical Documentaries", "Rockumentary", "media studies", "media satire"], 
    "Drama": ["Melodrama", "Tragedy", "Tragicomedy", "Docudrama", "Family drama", "Comedy-drama", 'illnesses & disabilities'], 
    "Family": ["Family Film"],
    "Fiction": ["fairy tail", "fictional film", "medical fiction", "mythological fantasy", "sci fi pictures", 'sci fi pictures original films', "sword and sorcery films", "sword and sandal", "time travel", "supernatural"],
    "Gross out": ["Gross-out Film"],
    "Historical": ["history", "Biographical", "British History", "British Empire Film", "British new wave", "Historical Epic", "Cold War", "Historical Documentaries"],
    "Monster": ["Creature Movie", "Monster Movie", "Creature"],
    "Horror": ["Natural Horror film", "Sci-fi Horror", "road-horror"],
    "Indie": ["Indie"],
    "Language & Literature": ["& Literature"], 
    "LGBT": ["New Queer Cinema", "Gay", "Homoeroticism"],
    "Old movies": ["Silent film", "black-and-white"],
    "Parody": [],
    "Political cinema": ["Political Documentary"],
    "Prison": ["Prison film", "Women in prison film"],
    "Master Criminal Films": ["Heist", "Gangster Film"],
    "Music": ["Concert film", "Film-Opera", "Operetta", "Punk rock", "Rockumentary", "Space opera", "Singing cowboy", "Hip hop movies", "Breakdance", "Dance", "musical"],
    "For children": ["Children's film", "children's", "children's entertainment", "children's fantasy", "children's/family"],
    "Religious Film": ["Christian film", "Cult"],
    "Romance": ["Romance film", "romantic fantasy"],
    "Sport": ["baseball", "boxing", "sports"],
    "Super hero": ["superhero movie", "superhero"],
    "Social issues": ["Social problem film"], 
    "Television Movie": ["TV Movie"],
    "Thriller": ["erotic thriller", "film à clef", "mystery", "suspense"],
    "War film": ["war", "War effort", "The Netherlands in World War II", "Gulf War", "Combat Films", "movies about gladiators", "anti-war", "anti-war film", "nuclear warfare", "foreign legion"],
    "Western": ["Singing cowboy", "Westerns", "B-Western", "Epic Western", "Spaghetti Western"],
    "World cinema": ["Chinese Movies", "Japanese Movies", "Latino", "americana", "bengali cinema", "filipino movies"],
    "Zombie": ["Zombie film", "Z-movie", "z movie"]
}
## To discuss: "world cinema"--> keep or remove? diversity?


##2nd step: Revert the current dictionnary, easier to manipulate

dict_genre_revert = {}
for key, value in dict_genre.items():
    key = key.lower() #passage en minuscule
    for v in value:
        v = v.lower()
        dict_genre_revert[v] = key
    dict_genre_revert[key] = key


##3rd step= create a new set with the new genre

def get_genre(set_original_genres):
    set_new_genres = set()
    for g in set_original_genres:
        g = g.lower()
        temp = True
        for s_g in g.split(" "): 
            if s_g in dict_genre_revert.keys():
                set_new_genres.add(dict_genre_revert[s_g])
                temp = False
        if temp:
            if g in dict_genre_revert.keys():
                set_new_genres.add(dict_genre_revert[g])
    return set_new_genres

# Test sur movie_English_only

df_test= movie_English_only.copy(deep=True)

## Changer les str de genre en dict + renvoyer set des values: /!\ interessant pour d'autres colonnes!!!
pd.set_option('mode.chained_assignment', None) #enlever les warnings de changement de valeur

for i in df_test.index:
    d=json.loads(df_test["Movie_genres"][i])
    df_test["Movie_genres"][i]=set(d.values())

## Changer les genres originaux en nouveux genres simplifiés
for i in df_test.index:
    df_test["Movie_genres"][i] = get_genre(df_test["Movie_genres"][i])

print(get_genre(df_test["Movie_genres"][1]))
print(movie_English_only["Movie_genres"][1])


{'thriller', 'drama', 'crime', 'historical'}
{"/m/02n4kr": "Mystery", "/m/03bxz7": "Biographical film", "/m/07s9rl0": "Drama", "/m/0hj3n01": "Crime Drama"}


In [162]:
df_test.head(25)

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{horror, adventure, thriller, action, fiction,..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{thriller, drama, crime, historical}"
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}",{thriller}
6,18998739,/m/04jcqvw,The Sorcerer's Apprentice,2002,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/0hzlz"": ""South Africa""}","{family, adventure, world cinema}"
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{music, comedy, old movies}"
9,2345652,/m/075f66,City of the Dead,1960,NaN,76.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{horror, fiction}"
10,175026,/m/017n1p,Sarah and Son,1930,NaN,86.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{old movies, drama}"
12,6631279,/m/0gffwj,Little city,1997-04-04,NaN,93.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{comedy, drama, romance}"
13,171005,/m/016ywb,Henry V,1989-11-08,10161099.0,137.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{action, drama, war film}"
17,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{music, drama, for children, comedy, family}"
